In [1]:
import pyodim
import hdr, hsda, common

imports static in hsda


In [2]:
filename = "/home/jsoderho/Downloads/66_20231110_063000.pvol.h5"
radar_datasets = pyodim.read_odim(filename, lazy_load=False)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
for sweep in range(len(radar_datasets)):
    #add HCA data
    hca_meta = common.get_odim_ncar_hca(radar_datasets[sweep]['elevation'].data[0], filename, radar_datasets[sweep]['DBZH'].shape, skip_birdbath=True)
    radar_datasets[sweep] = radar_datasets[sweep].merge(
        {'hca': (("azimuth", "range"), hca_meta['data']) })
    
    radar_datasets[sweep] = common.add_pyodim_sweep_metadata(radar_datasets[sweep], 'hca', hca_meta)
    #hail retrievals
    hdr_meta, hdr_size_meta = hdr.main(radar_datasets[sweep]['DBZH'].values,
                                       radar_datasets[sweep]['ZDR'].values)
    hsda_meta = hsda.main(radar_datasets[sweep]['DBZH'].values,
                                        radar_datasets[sweep]['ZDR'].values,
                                        radar_datasets[sweep]['RHOHV'].values,
                                        radar_datasets[sweep]['hca'].values,
                                        radar_datasets[sweep]['z'].values,
                                        [4000,5500],
                                        [6, 7, 8])
    #add new fields
    radar_datasets[sweep] = radar_datasets[sweep].merge(
        {'hdr': (("azimuth", "range"), hdr_meta['data']),
         'hdr_size': (("azimuth", "range"), hdr_size_meta['data']),
         'hsda': (("azimuth", "range"), hsda_meta['data']) })
    #update metadata for new fields
    radar_datasets[sweep] = common.add_pyodim_sweep_metadata(radar_datasets[sweep], 'hdr', hdr_meta)
    radar_datasets[sweep] = common.add_pyodim_sweep_metadata(radar_datasets[sweep], 'hdr_size', hdr_meta)
    radar_datasets[sweep] = common.add_pyodim_sweep_metadata(radar_datasets[sweep], 'hsda', hsda_meta)
